In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.patches as patches

import sncp_algorithm as algo

from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions


import logging
logger = logging.getLogger("root")


class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()


logger.addFilter(CheckTypesFilter())

In [ ]:
data = np.concatenate([
    tfd.StudentT(5, 0, 1).sample(100) - 5,
    tfd.StudentT(5, 0, 1).sample(100) +5
])
data = np.sort(data)
plt.hist(data)
plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import pairwise_distances
from scipy.stats import gaussian_kde

from scipy.signal import argrelextrema

grid = np.linspace(0, 20, 500)
dists = np.sort(pairwise_distances(data.reshape(-1, 1)).reshape(-1,))
kde_dens = gaussian_kde(dists)
eval_dens = kde_dens.pdf(grid)
argmins = argrelextrema(eval_dens, np.less)[0]

plt.plot(grid, eval_dens, color="k")
plt.vlines(grid[argmins[0]], ymin=0, ymax=1, color="r")
plt.ylim((0, 0.2))
plt.title(r"Pairwise Distances - $t$ Data", fontsize=18)
plt.savefig("t_distances.pdf", bbox_inches="tight")

In [ ]:
data = np.sort(data)

In [ ]:
from sncp_state import State, Prior

In [ ]:
prior = Prior(
  alpha = 2.0,
  big_mean = 0,
  big_var = np.var(data) * 5,
  gamma = 0.5,
  jump_a = 1.0,
  jump_b = 10.0,
  var_a = 3.0,
  var_b = 3.0
)

n_atoms = 5

alloc_atoms = np.hstack([np.random.normal(0, np.std(data) * 3, size=(n_atoms, 1)),
                           np.ones((n_atoms, 1)) * 1])
non_alloc_atoms = np.hstack([np.random.normal(0, np.std(data) * 3, size=(n_atoms, 1)),
                               np.ones((n_atoms, 1))])

state = State(
  iter = 0,
  clus = np.random.choice(np.arange(n_atoms), len(data)),
  alloc_atoms = alloc_atoms,
  non_alloc_atoms = non_alloc_atoms,
  alloc_jumps = np.ones(n_atoms),
  non_alloc_jumps = np.ones(n_atoms),
  u = 5,
  latent_centers = np.array([-5, 5]),
  t_vals = np.random.choice(np.arange(2), 2 * n_atoms)
)

In [ ]:
for i in range(1000):
    if (i % 100) == 0:
        print("\r{0} / {1}".format(i, 5000), flush=True, end=" ")
    state = algo.step(data, state, prior)

In [ ]:
states = [state]
for i in range(1000):
    if (i % 100) == 0:
        print("\r{0} / {1}".format(i, 5000), flush=True, end=" ")
    state = algo.step(data, state, prior)
    states.append(state)

In [ ]:
np.unique(state.latent_centers[state.active_t_vals])

In [ ]:
import pickle

with open("t_data/t_simulation_mcmc_sncp.pickle", "wb") as fp:
   pickle.dump(states, fp)

In [ ]:
with open("t_data/t_simulation_mcmc_sncp.pickle", "rb") as fp:
   states = pickle.load(fp)

In [ ]:
grid = np.linspace(-15, 15, 1000)


def get_dens(state, grid):
    eval_comps = tfd.Normal(state.alloc_atoms[:, 0], np.sqrt(state.alloc_atoms[:, 1])).prob(grid[:, np.newaxis]) 
    weights = state.alloc_jumps
    weights /= np.sum(weights)
    dens = np.sum(eval_comps * weights, axis=1)
    return dens

In [ ]:
from joblib import Parallel, delayed


eval_dens = Parallel(n_jobs=4)(
    delayed(lambda x: get_dens(x, grid))(x) for x in states[1:])

In [ ]:
eval_dens = np.vstack(eval_dens)
eval_dens.shape

In [ ]:
from bayesmixpy import build_bayesmix, run_mcmc

build_bayesmix(4)

In [ ]:
mfm_params = """
fixed_value {
    lambda: 4.0
    gamma: 1.0
}
"""

g0_params = """
fixed_values {
    mean: 0.0
    var_scaling: 0.1
    shape: 2.0
    scale: 2.0
}
"""

algo_params = """
    algo_id: "Neal2"
    rng_seed: 20201124
    iterations: 10000
    burnin: 5000
    init_num_clusters: 6
    neal8_n_aux: 10
"""

data = np.concatenate([
    tfd.StudentT(3, 0, 1).sample(100) - 5,
    tfd.StudentT(3, 0, 1).sample(100) +5
])
data = np.sort(data)


log_dens_dp, numcluschain_dp, cluschain_dp, bestclus_dp = run_mcmc(
    "NNIG", "MFM", data, g0_params, mfm_params, algo_params, 
    dens_grid=grid, return_clusters=True, return_num_clusters=True,
    return_best_clus=False, out_dir="t_data/")

In [ ]:
figure = plt.figure(figsize=(5, 5))
plt.imshow(get_psm(cluschain_dp, False), cmap="Greys")
plt.title("PSM - IFPP", fontsize=18)
plt.savefig("sncp_simulation_a.pdf",bbox_inches="tight")

In [ ]:
numcluschain_dp = np.loadtxt("./t_data/n_clus.csv")
log_dens_dp = np.loadtxt("./t_data/eval_dens.csv", delimiter=",")
cluschain_dp = np.loadtxt("./t_data/clus.csv", delimiter=",").astype(int)


In [ ]:
from scipy.cluster.hierarchy import linkage

def get_psm(clus_chain, sort=True):
    ndata = clus_chain.shape[1]
    out = np.zeros((ndata, ndata))
    for i in range(ndata):
        for j in range(i):
            out[i, j] = out[j, i] = np.mean(clus_chain[:, i] == clus_chain[:, j])
            
    out = out + np.eye(ndata)
    
    if sort:
        y = 1 - out[np.triu_indices(len(out), k=1)]
        Z = linkage(y, method='single', optimal_ordering=True)
        perm = np.ravel(Z[:, :2]).astype(np.int32)
        perm = perm[perm < len(out)]
        out = out[perm][:, perm]
    
    return out

def get_bars_heights(x, clus):
    counts = np.sum(clus == x[:, np.newaxis], axis=1)
    return counts / np.sum(counts)

In [ ]:
import pickle 

with open("t_data/t_out_repulsive.pickle", "rb") as fp:
    repulsive_fit = pickle.load(fp)

repulsive_fit.keys()

In [ ]:
figure = plt.figure(figsize=(5, 5))
cluschain_sncp = np.vstack([x.t_vals[x.clus] for x in states])
plt.imshow(get_psm(cluschain_sncp, False), cmap="Greys")
plt.title("PSM - SNCP", fontsize=18)
plt.savefig("sncp_simulation_b.pdf",bbox_inches="tight")

In [ ]:
figure = plt.figure(figsize=(5, 5))
x = np.arange(1, 12)

cluschain_sncp = np.vstack([x.t_vals[x.clus] for x in states])
nclus2 = np.array([len(np.unique(x)) for x in cluschain_sncp])
h2 = get_bars_heights(x, nclus2)
plt.plot(x, h2, "X--", lw=2, markersize=10, color="blue", label="SNCP")

h2 = get_bars_heights(x, repulsive_fit["nclus_chain"])
plt.plot(x, h2, linestyle="dotted", lw=2, markersize=10, color="orange", label="DPP")

h3 = get_bars_heights(x, numcluschain_dp)
plt.plot(x, h3, "+:", lw=2, markersize=10, markeredgewidth = 3, color="forestgreen", label="IFPP")
plt.legend(fontsize=18)
plt.title("# Clusters", fontsize=18)
plt.xlabel("c", fontsize=16)
plt.ylabel("P($K_n$ = c)", fontsize=16)
plt.xticks(x)
plt.savefig("sncp_simulation_c.pdf",bbox_inches="tight")

In [ ]:
true_dens = 0.5 * tfd.StudentT(3, -5, 1).prob(grid) + 0.5 * tfd.StudentT(3, 5, 1).prob(grid)

In [ ]:
figure = plt.figure(figsize=(5, 5))

plt.plot(grid, true_dens, color="red", label="True")
plt.plot(grid, np.mean(eval_dens, axis=0), color="blue", label="SNCP")
plt.plot(repulsive_fit["xgrid"], np.mean(repulsive_fit["dens_dpp"], axis=0),
          color="orange", label="DPP")

plt.plot(grid, np.mean(np.exp(log_dens_dp), axis=0),  color="forestgreen", label="IFPP")
plt.xlim(-12, 12)
plt.legend(fontsize=18, ncol=2, loc="lower left", bbox_to_anchor=(0.05, -0.3))
plt.title("Density Estimate", fontsize=19)
plt.savefig("sncp_simulation_d.pdf", bbox_inches="tight")

# Data From Miller and Dunson

In [ ]:
import pickle

In [ ]:
with open("contaminated_data/miller_out_repulsive.pickle", "rb") as fp:
    repulsive_fit = pickle.load(fp)

repulsive_fit.keys()

In [ ]:
with open("contaminated_data/miller_data.pickle", "rb") as fp:
    miller_data = pickle.load(fp)

miller_data.keys()

In [ ]:
data = miller_data["data"]
grid = repulsive_fit["xgrid"]

In [ ]:
import seaborn as sns
from sklearn.metrics import pairwise_distances
from scipy.stats import gaussian_kde

from scipy.signal import argrelextrema

grid = np.linspace(0, 15, 500)
dists = np.sort(pairwise_distances(data.reshape(-1, 1)).reshape(-1,))
kde_dens = gaussian_kde(dists)
eval_dens = kde_dens.pdf(grid)
argmins = argrelextrema(eval_dens, np.less)[0]

plt.plot(grid, eval_dens, color="k")
plt.vlines(grid[argmins[0]], ymin=0, ymax=1, color="r")
plt.ylim((0, 0.25))
plt.title(r"Pairwise Distances - Contaminated Data", fontsize=18)
plt.savefig("plots/miller_distances.pdf", bbox_inches="tight")

In [ ]:
from bayesmixpy import build_bayesmix, run_mcmc

dp_params = """
fixed_value {
    totalmass: 1.0
}
"""

mfm_params = """
fixed_value {
    lambda: 4.0
    gamma: 1.0
}
"""

g0_params = """
fixed_values {
    mean: 0.0
    var_scaling: 0.1
    shape: 2.0
    scale: 2.0
}
"""

algo_params = """
    algo_id: "Neal2"
    rng_seed: 20201124
    iterations: 2000
    burnin: 1000
    init_num_clusters: 3
"""

log_dens_dp, numcluschain_dp, cluschain_dp, bestclus_dp = run_mcmc(
    "NNIG", "MFM", miller_data["data"], g0_params, mfm_params, algo_params, 
    dens_grid=grid, return_clusters=True, return_num_clusters=True,
    return_best_clus=True, out_dir="./miller_data/ifpp")

In [ ]:
prior = Prior(
  alpha = 1,
  big_mean = np.mean(data),
  big_var = np.var(data) * 5,
  gamma = 1.0,
  jump_a = 1.0,
  jump_b = 10.0,
  var_a = 6.0,
  var_b = 3.0
)

n_atoms = 5

alloc_atoms = np.hstack([np.random.normal(0, np.std(data) * 3, size=(n_atoms, 1)),
                           np.ones((n_atoms, 1)) * 1])
non_alloc_atoms = np.hstack([np.random.normal(0, np.std(data) * 3, size=(n_atoms, 1)),
                               np.ones((n_atoms, 1))])

state = State(
  iter = 0,
  clus = np.random.choice(np.arange(n_atoms), len(data)),
  alloc_atoms = alloc_atoms,
  non_alloc_atoms = non_alloc_atoms,
  alloc_jumps = np.ones(n_atoms),
  non_alloc_jumps = np.ones(n_atoms),
  u = 5,
  latent_centers = np.array([-5, -1,  0,  2, 7]),
  t_vals = np.random.choice(np.arange(5), 2 * n_atoms)
)

for i in range(1000):
    if (i % 100) == 0:
        print("\r{0} / {1}".format(i, 1000), flush=True, end=" ")
    state = algo.step(data, state, prior)

states = []
for i in range(1000):
    if (i % 100) == 0:
        print("\r{0} / {1}".format(i, 1000), flush=True, end=" ")
    state = algo.step(data, state, prior)
    states.append(state)

In [ ]:
import pickle

with open("contaminated_data/miller_simulation_mcmc_sncp.pickle", "wb") as fp:
  pickle.dump(states, fp)

In [ ]:
eval_dens = Parallel(n_jobs=4)(
    delayed(lambda x: get_dens(x, grid))(x) for x in states)
eval_dens = np.vstack(eval_dens)

In [ ]:
figure = plt.figure(figsize=(5, 5))
x = np.arange(1, 12)

cluschain_sncp = np.vstack([x.t_vals[x.clus] for x in states])
nclus2 = np.array([len(np.unique(x)) for x in cluschain_sncp])

h2 = get_bars_heights(x, nclus2)
plt.plot(x, h2, "X--", lw=2, markersize=10, color="blue", label="SNCP")

h2 = get_bars_heights(x, repulsive_fit["nclus_chain"])
plt.plot(x, h2, marker="o", linestyle="dotted", lw=2, markersize=10, color="orange", label="DPP")

h3 = get_bars_heights(x, numcluschain_dp)
plt.plot(x, h3, "+:", lw=2, markersize=10, markeredgewidth = 3, color="forestgreen", label="IFPP")
plt.legend(fontsize=18, ncol=2, loc="lower left", bbox_to_anchor=(0.05, -0.3))
plt.title("# Clusters", fontsize=18)
# plt.xlabel("c", fontsize=16)
plt.ylabel("P($K_n$ = c)", fontsize=16)
plt.xticks(x)
# plt.show()
plt.savefig("miller_simulation_miller_nclus.pdf",bbox_inches="tight")

In [ ]:
true_f = np.zeros_like(grid)
for w, mu, s in zip(miller_data["w_t"], miller_data["true_mu"], miller_data["true_sigma"]):
    true_f += w * tfd.Normal(mu, s).prob(grid)

In [ ]:
figure = plt.figure(figsize=(5, 5))

plt.plot(grid, true_f, color="red", label="True")
plt.plot(grid, np.mean(eval_dens, axis=0), color="blue", label="SNCP")
plt.plot(repulsive_fit["xgrid"], np.mean(repulsive_fit["dens_dpp"], axis=0),
          color="orange", label="DPP")

# plt.plot(grid, np.mean(np.exp(log_dens_dp), axis=0),  color="forestgreen", label="IFPP")
plt.legend(fontsize=18, ncol=2, loc="lower left", bbox_to_anchor=(0.05, -0.3))
plt.title("Density Estimate", fontsize=19)
# plt.savefig("sncp_simulation_miller_density.pdf", bbox_inches="tight")